# Boxing and Unboxing - Good vs Bad Examples

This notebook demonstrates **meaningful** vs **problematic** implementations of boxing and unboxing in C#.

## What Are Boxing and Unboxing?

**Boxing** is the process of converting a value type to a reference type (specifically to `object` or an interface).
**Unboxing** is the explicit conversion from a reference type back to a value type.

### Key Concepts:
- **Value types** live on the stack (int, double, struct, enum)
- **Reference types** live on the heap (class, object, string, arrays)
- **Boxing** allocates memory on the heap and copies the value
- **Unboxing** extracts the value from the heap back to the stack
- **Performance cost** - boxing/unboxing involves memory allocation and copying

### When Boxing Occurs:
1. Assigning value type to `object` or interface
2. Using non-generic collections (ArrayList, Hashtable)
3. Calling virtual methods on value types
4. Using reflection with value types
5. String concatenation with + operator (in some cases)

### ✅ Good Examples - Appropriate Boxing/Unboxing Usage

In [ ]:
// ✅ GOOD: Boxing when working with heterogeneous data (necessary)
public class ConfigurationManager
{
    private readonly Dictionary<string, object> _settings = new();
    
    // Boxing is necessary here to store different value types
    public void SetSetting<T>(string key, T value) where T : struct
    {
        _settings[key] = value; // Boxing occurs here - but it's necessary
        Console.WriteLine($"Stored {typeof(T).Name} value: {value} (boxed to object)");
    }
    
    // Unboxing with type safety
    public T GetSetting<T>(string key, T defaultValue = default) where T : struct
    {
        if (_settings.TryGetValue(key, out var value) && value is T typedValue)
        {
            Console.WriteLine($"Retrieved and unboxed {typeof(T).Name}: {typedValue}");
            return typedValue; // Unboxing occurs here
        }
        
        Console.WriteLine($"Key '{key}' not found, returning default: {defaultValue}");
        return defaultValue;
    }
    
    // Safe unboxing with validation
    public bool TryGetSetting<T>(string key, out T value) where T : struct
    {
        value = default;
        
        if (_settings.TryGetValue(key, out var objValue) && objValue is T)
        {
            value = (T)objValue; // Safe unboxing
            return true;
        }
        
        return false;
    }
    
    public void DisplayAllSettings()
    {
        Console.WriteLine("All settings:");
        foreach (var kvp in _settings)
        {
            // Boxing already occurred when storing, no additional boxing here
            Console.WriteLine($"  {kvp.Key}: {kvp.Value} (Type: {kvp.Value.GetType().Name})");
        }
    }
}

In [ ]:
// ✅ GOOD USAGE: Demonstrate necessary boxing for heterogeneous storage
var config = new ConfigurationManager();

// These operations require boxing because we're storing different types
config.SetSetting("MaxConnections", 100);
config.SetSetting("Timeout", 30.5);
config.SetSetting("IsEnabled", true);
config.SetSetting("UserId", new Guid("12345678-1234-1234-1234-123456789012"));

Console.WriteLine();
config.DisplayAllSettings();

Console.WriteLine();
// Safe retrieval with unboxing
var maxConn = config.GetSetting("MaxConnections", 50);
var timeout = config.GetSetting("Timeout", 15.0);
var isEnabled = config.GetSetting("IsEnabled", false);

Console.WriteLine($"Max connections: {maxConn}");
Console.WriteLine($"Timeout: {timeout}");
Console.WriteLine($"Is enabled: {isEnabled}");

Console.WriteLine();
// Try getting non-existent setting
if (config.TryGetSetting<int>("NonExistent", out var nonExistentValue))
{
    Console.WriteLine($"Found: {nonExistentValue}");
}
else
{
    Console.WriteLine("Setting not found (safe unboxing prevented error)");
}
Console.WriteLine();

In [ ]:
// ✅ GOOD: Boxing when implementing interfaces (sometimes necessary)
public struct Point : IComparable<Point>, IEquatable<Point>
{
    public int X { get; }
    public int Y { get; }
    
    public Point(int x, int y)
    {
        X = x;
        Y = y;
    }
    
    // This will NOT cause boxing when called on Point directly
    public int CompareTo(Point other)
    {
        var result = X.CompareTo(other.X);
        return result != 0 ? result : Y.CompareTo(other.Y);
    }
    
    public bool Equals(Point other)
    {
        return X == other.X && Y == other.Y;
    }
    
    public override bool Equals(object obj)
    {
        return obj is Point other && Equals(other);
    }
    
    public override int GetHashCode()
    {
        return HashCode.Combine(X, Y);
    }
    
    public override string ToString()
    {
        return $"({X}, {Y})";
    }
}

// Helper class to demonstrate interface usage
public class GeometryProcessor
{
    // This method will cause boxing when Point is passed
    public static void ProcessComparable(IComparable comparable, object compareWith)
    {
        Console.WriteLine($"Processing: {comparable} (Type: {comparable.GetType().Name})");
        
        if (compareWith != null && comparable.GetType() == compareWith.GetType())
        {
            var result = comparable.CompareTo(compareWith);
            var comparison = result switch
            {
                < 0 => "less than",
                0 => "equal to",
                > 0 => "greater than"
            };
            Console.WriteLine($"  Result: {comparable} is {comparison} {compareWith}");
        }
    }
    
    // Generic version avoids boxing
    public static void ProcessComparableGeneric<T>(T item, T compareWith) where T : IComparable<T>
    {
        Console.WriteLine($"Processing (generic): {item} (Type: {typeof(T).Name})");
        
        var result = item.CompareTo(compareWith);
        var comparison = result switch
        {
            < 0 => "less than",
            0 => "equal to",
            > 0 => "greater than"
        };
        Console.WriteLine($"  Result (no boxing): {item} is {comparison} {compareWith}");
    }
}

In [ ]:
// Usage examples showing boxing vs non-boxing scenarios
var point1 = new Point(3, 4);
var point2 = new Point(1, 2);
var point3 = new Point(3, 4);

Console.WriteLine("=== Direct struct usage (NO boxing) ===");
Console.WriteLine($"Point1: {point1}");
Console.WriteLine($"Point1 == Point3: {point1.Equals(point3)}");
Console.WriteLine($"Point1.CompareTo(Point2): {point1.CompareTo(point2)}");
Console.WriteLine();

Console.WriteLine("=== Interface usage (BOXING occurs) ===");
// Boxing happens here because we're treating Point as IComparable
GeometryProcessor.ProcessComparable(point1, point2);
GeometryProcessor.ProcessComparable(point1, point3);
Console.WriteLine();

Console.WriteLine("=== Generic interface usage (NO boxing) ===");
// No boxing with generic constraints
GeometryProcessor.ProcessComparableGeneric(point1, point2);
GeometryProcessor.ProcessComparableGeneric(point1, point3);
Console.WriteLine();

Console.WriteLine("=== Collection scenarios ===");
// Boxing when storing in object collection
object[] objects = { point1, point2, point3 };
Console.WriteLine("Stored in object array (boxing occurred):");
foreach (var obj in objects)
{
    Console.WriteLine($"  {obj} (runtime type: {obj.GetType().Name})");
}

// No boxing with generic collection
var points = new List<Point> { point1, point2, point3 };
Console.WriteLine("\nStored in List<Point> (no boxing):");
foreach (var point in points)
{
    Console.WriteLine($"  {point}");
}
Console.WriteLine();

In [ ]:
// ✅ GOOD: Working with legacy APIs that require boxing
public class LegacyDataProcessor
{
    // Simulating old API that uses ArrayList (legacy, but sometimes unavoidable)
    public static ArrayList ProcessLegacyData(params object[] values)
    {
        var result = new ArrayList();
        
        Console.WriteLine("Processing legacy data (boxing required):");
        foreach (var value in values)
        {
            // Values are already boxed from params object[]
            result.Add(value);
            Console.WriteLine($"  Added: {value} (Type: {value.GetType().Name})");
        }
        
        return result;
    }
    
    // Safe unboxing from legacy collection
    public static T SafeUnbox<T>(object boxedValue, T defaultValue = default) where T : struct
    {
        try
        {
            if (boxedValue is T value)
            {
                Console.WriteLine($"Successfully unboxed {typeof(T).Name}: {value}");
                return value;
            }
            else
            {
                Console.WriteLine($"Type mismatch: expected {typeof(T).Name}, got {boxedValue?.GetType().Name ?? "null"}");
                return defaultValue;
            }
        }
        catch (InvalidCastException ex)
        {
            Console.WriteLine($"Unboxing failed: {ex.Message}");
            return defaultValue;
        }
    }
    
    // Modern alternative that avoids boxing
    public static List<T> ProcessModernData<T>(params T[] values) where T : struct
    {
        var result = new List<T>();
        
        Console.WriteLine($"Processing modern data (no boxing for {typeof(T).Name}):");
        foreach (var value in values)
        {
            result.Add(value); // No boxing
            Console.WriteLine($"  Added: {value}");
        }
        
        return result;
    }
}

In [ ]:
// Usage examples for legacy API scenarios
Console.WriteLine("=== Legacy API (boxing unavoidable) ===");
var legacyResult = LegacyDataProcessor.ProcessLegacyData(42, 3.14, true, 'A');

Console.WriteLine("\nSafe unboxing from legacy collection:");
foreach (var item in legacyResult)
{
    // Try to unbox as different types
    if (item is int)
    {
        var intValue = LegacyDataProcessor.SafeUnbox<int>(item);
        Console.WriteLine($"  Found int: {intValue}");
    }
    else if (item is double)
    {
        var doubleValue = LegacyDataProcessor.SafeUnbox<double>(item);
        Console.WriteLine($"  Found double: {doubleValue}");
    }
    else if (item is bool)
    {
        var boolValue = LegacyDataProcessor.SafeUnbox<bool>(item);
        Console.WriteLine($"  Found bool: {boolValue}");
    }
    else if (item is char)
    {
        var charValue = LegacyDataProcessor.SafeUnbox<char>(item);
        Console.WriteLine($"  Found char: {charValue}");
    }
}

Console.WriteLine("\n=== Modern API (no boxing) ===");
var modernInts = LegacyDataProcessor.ProcessModernData(1, 2, 3, 4, 5);
var modernDoubles = LegacyDataProcessor.ProcessModernData(1.1, 2.2, 3.3);

Console.WriteLine($"Modern ints: [{string.Join(", ", modernInts)}]");
Console.WriteLine($"Modern doubles: [{string.Join(", ", modernDoubles)}]");
Console.WriteLine();

### ❌ Bad Examples - Unnecessary Boxing/Unboxing

In [ ]:
// ❌ BAD: Unnecessary boxing in collections
public class BadCollectionUsage
{
    // ❌ BAD: Using non-generic collections when generic ones would work
    public static ArrayList CreateNumberListBadly()
    {
        var list = new ArrayList(); // Stores objects, causes boxing
        
        for (int i = 0; i < 5; i++)
        {
            list.Add(i); // Boxing occurs here - int → object
        }
        
        return list;
    }
    
    // ❌ BAD: Unnecessary unboxing when processing
    public static int SumArrayListBadly(ArrayList numbers)
    {
        int sum = 0;
        foreach (object obj in numbers)
        {
            sum += (int)obj; // Unboxing required - object → int
        }
        return sum;
    }
    
    // ❌ BAD: Using Hashtable instead of Dictionary
    public static Hashtable CreateMapBadly()
    {
        var map = new Hashtable(); // Keys and values are objects
        
        map[1] = 100;    // Both key and value are boxed
        map[2] = 200;    // int → object for both
        map[3] = 300;
        
        return map;
    }
    
    // ❌ BETTER: Generic versions (no boxing)
    public static List<int> CreateNumberListGood()
    {
        var list = new List<int>(); // Type-safe, no boxing
        
        for (int i = 0; i < 5; i++)
        {
            list.Add(i); // No boxing - int stays int
        }
        
        return list;
    }
    
    public static int SumListGood(List<int> numbers)
    {
        int sum = 0;
        foreach (int number in numbers) // No unboxing needed
        {
            sum += number;
        }
        return sum;
    }
    
    public static Dictionary<int, int> CreateMapGood()
    {
        var map = new Dictionary<int, int>(); // Type-safe
        
        map[1] = 100;    // No boxing
        map[2] = 200;
        map[3] = 300;
        
        return map;
    }
}

In [ ]:
// ❌ BAD: Excessive boxing in string operations
public class BadStringOperations
{
    // ❌ BAD: String concatenation causing boxing
    public static string FormatNumbersBadly(int[] numbers)
    {
        string result = "Numbers: ";
        
        foreach (int number in numbers)
        {
            result += number + ", "; // Boxing: int → object for concatenation
        }
        
        return result.TrimEnd(',', ' ');
    }
    
    // ❌ BAD: Using ToString() unnecessarily
    public static void PrintValuesBadly(params object[] values)
    {
        foreach (var value in values) // Values are already boxed from params
        {
            Console.WriteLine(value.ToString()); // Additional method call overhead
        }
    }
    
    // ✅ BETTER: String interpolation (compiler optimizes)
    public static string FormatNumbersGood(int[] numbers)
    {
        return $"Numbers: {string.Join(", ", numbers)}";
        // No boxing - compiler and Join method handle efficiently
    }
    
    // ✅ BETTER: Generic approach
    public static void PrintValuesGood<T>(params T[] values)
    {
        foreach (var value in values) // No boxing for value types
        {
            Console.WriteLine(value); // Direct conversion
        }
    }
    
    // ✅ BETTER: StringBuilder for multiple concatenations
    public static string FormatNumbersEfficient(int[] numbers)
    {
        var sb = new StringBuilder("Numbers: ");
        
        for (int i = 0; i < numbers.Length; i++)
        {
            if (i > 0) sb.Append(", ");
            sb.Append(numbers[i]); // StringBuilder.Append(int) - no boxing
        }
        
        return sb.ToString();
    }
}

In [ ]:
// ❌ BAD: Dangerous unboxing without validation
public class DangerousUnboxing
{
    // ❌ BAD: Unsafe unboxing that can throw exceptions
    public static void ProcessObjectUnsafely(object obj)
    {
        // This will throw InvalidCastException if obj is not an int
        int number = (int)obj; // Dangerous unboxing
        Console.WriteLine($"Number: {number}");
    }
    
    // ❌ BAD: Wrong type assumption
    public static double CalculateAverage(ArrayList numbers)
    {
        double sum = 0;
        foreach (object obj in numbers)
        {
            // Assumes all objects are numbers - dangerous!
            if (obj is int intVal)
                sum += intVal;
            else if (obj is double doubleVal)
                sum += doubleVal;
            else
                sum += (double)obj; // This could throw!
        }
        return sum / numbers.Count;
    }
    
    // ✅ BETTER: Safe unboxing with validation
    public static void ProcessObjectSafely(object obj)
    {
        if (obj is int number)
        {
            Console.WriteLine($"Number: {number}");
        }
        else
        {
            Console.WriteLine($"Not an integer: {obj?.GetType().Name ?? "null"}");
        }
    }
    
    // ✅ BETTER: Generic approach avoids boxing entirely
    public static double CalculateAverageGeneric<T>(IEnumerable<T> numbers) 
        where T : struct, IConvertible
    {
        double sum = 0;
        int count = 0;
        
        foreach (var number in numbers)
        {
            sum += number.ToDouble(null); // No boxing
            count++;
        }
        
        return count > 0 ? sum / count : 0;
    }
}

In [ ]:
// ❌ BAD USAGE: Demonstrate performance and safety issues

Console.WriteLine("❌ Examples of what NOT to do:");
Console.WriteLine();

// Bad collection usage
Console.WriteLine("=== Bad Collection Usage (Unnecessary Boxing) ===");
var badList = BadCollectionUsage.CreateNumberListBadly();
var badSum = BadCollectionUsage.SumArrayListBadly(badList);
Console.WriteLine($"ArrayList sum (with boxing/unboxing): {badSum}");

var goodList = BadCollectionUsage.CreateNumberListGood();
var goodSum = BadCollectionUsage.SumListGood(goodList);
Console.WriteLine($"List<int> sum (no boxing): {goodSum}");
Console.WriteLine();

// Bad string operations
Console.WriteLine("=== Bad String Operations (Unnecessary Boxing) ===");
var numbers = new[] { 1, 2, 3, 4, 5 };

var badFormat = BadStringOperations.FormatNumbersBadly(numbers);
Console.WriteLine($"Bad formatting (boxing): {badFormat}");

var goodFormat = BadStringOperations.FormatNumbersGood(numbers);
Console.WriteLine($"Good formatting (no boxing): {goodFormat}");

var efficientFormat = BadStringOperations.FormatNumbersEfficient(numbers);
Console.WriteLine($"Efficient formatting (StringBuilder): {efficientFormat}");
Console.WriteLine();

// Unsafe unboxing
Console.WriteLine("=== Unsafe vs Safe Unboxing ===");
object[] mixedData = { 42, "not a number", 3.14, true };

foreach (var item in mixedData)
{
    Console.Write($"Processing {item}: ");
    
    try
    {
        DangerousUnboxing.ProcessObjectSafely(item); // Safe approach
    }
    catch (Exception ex)
    {
        Console.WriteLine($"Error: {ex.Message}");
    }
}
Console.WriteLine();

// Performance implications
Console.WriteLine("=== Performance Implications ===");
Console.WriteLine("❌ ArrayList: Boxing every int (heap allocation + copy)");
Console.WriteLine("✅ List<int>: No boxing (direct storage)");
Console.WriteLine("❌ String concatenation with +: Boxing for ToString()");
Console.WriteLine("✅ String interpolation: Compiler optimized");
Console.WriteLine("❌ Unsafe casting: Runtime exceptions possible");
Console.WriteLine("✅ Pattern matching: Safe type checking");
Console.WriteLine();

## Performance Analysis and Measurement

In [ ]:
// ✅ GOOD: Measuring boxing performance impact
using System.Diagnostics;

public class BoxingPerformanceAnalysis
{
    public static void CompareCollectionPerformance(int iterations = 100000)
    {
        Console.WriteLine($"Performance comparison with {iterations:N0} iterations:");
        Console.WriteLine();
        
        // Test ArrayList (with boxing)
        var sw = Stopwatch.StartNew();
        var arrayList = new ArrayList();
        
        for (int i = 0; i < iterations; i++)
        {
            arrayList.Add(i); // Boxing occurs
        }
        
        sw.Stop();
        var arrayListTime = sw.ElapsedMilliseconds;
        var arrayListMemory = GC.GetTotalMemory(false);
        
        // Force garbage collection to measure memory more accurately
        GC.Collect();
        GC.WaitForPendingFinalizers();
        GC.Collect();
        
        var baseMemory = GC.GetTotalMemory(false);
        
        // Test List<int> (no boxing)
        sw.Restart();
        var genericList = new List<int>();
        
        for (int i = 0; i < iterations; i++)
        {
            genericList.Add(i); // No boxing
        }
        
        sw.Stop();
        var genericListTime = sw.ElapsedMilliseconds;
        var genericListMemory = GC.GetTotalMemory(false);
        
        Console.WriteLine($"ArrayList (with boxing):");
        Console.WriteLine($"  Time: {arrayListTime} ms");
        Console.WriteLine($"  Objects created: {iterations:N0} (each int was boxed)");
        Console.WriteLine();
        
        Console.WriteLine($"List<int> (no boxing):");
        Console.WriteLine($"  Time: {genericListTime} ms");
        Console.WriteLine($"  Objects created: 1 (just the list itself)");
        Console.WriteLine();
        
        var speedupFactor = arrayListTime > 0 ? (double)arrayListTime / genericListTime : 0;
        Console.WriteLine($"Performance improvement: {speedupFactor:F1}x faster");
        Console.WriteLine($"Memory efficiency: Significantly less memory allocation");
        Console.WriteLine();
        
        // Cleanup
        arrayList.Clear();
        genericList.Clear();
    }
    
    public static void DemonstrateStringBoxing()
    {
        const int iterations = 10000;
        Console.WriteLine($"String concatenation comparison with {iterations:N0} iterations:");
        
        // Bad: String concatenation with boxing
        var sw = Stopwatch.StartNew();
        string result1 = "";
        
        for (int i = 0; i < iterations; i++)
        {
            result1 += i + ","; // Boxing occurs for each i
        }
        
        sw.Stop();
        var badTime = sw.ElapsedMilliseconds;
        
        // Good: StringBuilder without boxing
        sw.Restart();
        var sb = new StringBuilder();
        
        for (int i = 0; i < iterations; i++)
        {
            sb.Append(i).Append(','); // No boxing
        }
        
        string result2 = sb.ToString();
        sw.Stop();
        var goodTime = sw.ElapsedMilliseconds;
        
        Console.WriteLine($"String concatenation (with boxing): {badTime} ms");
        Console.WriteLine($"StringBuilder (no boxing): {goodTime} ms");
        
        if (goodTime > 0)
        {
            Console.WriteLine($"Performance improvement: {(double)badTime / goodTime:F1}x faster");
        }
        
        Console.WriteLine($"Results equal: {result1.Length == result2.Length}");
        Console.WriteLine();
    }
}

In [ ]:
// Run performance analysis
Console.WriteLine("🔍 Boxing Performance Analysis");
Console.WriteLine("=" * 50);
Console.WriteLine();

// Collection performance
BoxingPerformanceAnalysis.CompareCollectionPerformance(50000);

// String concatenation performance
BoxingPerformanceAnalysis.DemonstrateStringBoxing();

Console.WriteLine("📊 Key Takeaways:");
Console.WriteLine("• Boxing creates objects on the heap (memory allocation)");
Console.WriteLine("• Each boxed value type becomes a separate object");
Console.WriteLine("• Unboxing requires type checking and copying");
Console.WriteLine("• Generic collections avoid boxing entirely");
Console.WriteLine("• StringBuilder methods are optimized for value types");
Console.WriteLine("• String interpolation is compiler-optimized");
Console.WriteLine();

## Real-World Boxing Scenarios

In [ ]:
// ✅ GOOD: Real-world scenarios where boxing is necessary or acceptable
public class RealWorldBoxingScenarios
{
    // Scenario 1: Event handling with different data types
    public class EventAggregator
    {
        private readonly Dictionary<Type, List<Action<object>>> _handlers = new();
        
        public void Subscribe<T>(Action<T> handler) where T : struct
        {
            var type = typeof(T);
            if (!_handlers.ContainsKey(type))
                _handlers[type] = new List<Action<object>>();
                
            // Boxing is necessary here to store different handler types
            _handlers[type].Add(obj => handler((T)obj));
        }
        
        public void Publish<T>(T eventData) where T : struct
        {
            var type = typeof(T);
            if (_handlers.TryGetValue(type, out var handlers))
            {
                // Boxing occurs here, but it's necessary for the event system
                object boxedData = eventData;
                foreach (var handler in handlers)
                {
                    handler(boxedData);
                }
            }
        }
    }
    
    // Scenario 2: Serialization where boxing may be necessary
    public class SimpleSerializer
    {
        public string SerializeValue(object value)
        {
            // Value types will be boxed when passed as object parameter
            return value switch
            {
                int i => $"int:{i}",
                double d => $"double:{d}",
                bool b => $"bool:{b}",
                string s => $"string:{s}",
                _ => $"object:{value}"
            };
        }
        
        public object DeserializeValue(string serialized)
        {
            var parts = serialized.Split(':', 2);
            if (parts.Length != 2) return serialized;
            
            return parts[0] switch
            {
                "int" => int.Parse(parts[1]),     // Boxing on return
                "double" => double.Parse(parts[1]), // Boxing on return
                "bool" => bool.Parse(parts[1]),   // Boxing on return
                "string" => parts[1],
                _ => serialized
            };
        }
    }
    
    // Scenario 3: Reflection scenarios
    public class PropertyInspector
    {
        public static void InspectObject(object obj)
        {
            var type = obj.GetType();
            Console.WriteLine($"Inspecting {type.Name}:");
            
            foreach (var prop in type.GetProperties())
            {
                try
                {
                    var value = prop.GetValue(obj); // Boxing occurs for value type properties
                    Console.WriteLine($"  {prop.Name}: {value} (Type: {value?.GetType().Name ?? "null"})");
                }
                catch (Exception ex)
                {
                    Console.WriteLine($"  {prop.Name}: Error - {ex.Message}");
                }
            }
        }
    }
}

In [ ]:
// Usage examples for real-world scenarios
Console.WriteLine("🌍 Real-World Boxing Scenarios");
Console.WriteLine("=" * 40);
Console.WriteLine();

// Event aggregator example
Console.WriteLine("=== Event System (Boxing Necessary) ===");
var eventAggregator = new RealWorldBoxingScenarios.EventAggregator();

// Subscribe to different event types
eventAggregator.Subscribe<int>(count => Console.WriteLine($"  Count event: {count}"));
eventAggregator.Subscribe<bool>(flag => Console.WriteLine($"  Flag event: {flag}"));
eventAggregator.Subscribe<double>(value => Console.WriteLine($"  Value event: {value:F2}"));

// Publish events (boxing occurs)
Console.WriteLine("Publishing events:");
eventAggregator.Publish(42);
eventAggregator.Publish(true);
eventAggregator.Publish(3.14159);
Console.WriteLine();

// Serialization example
Console.WriteLine("=== Serialization (Boxing on Interface) ===");
var serializer = new RealWorldBoxingScenarios.SimpleSerializer();

var values = new object[] { 42, 3.14, true, "hello" };
var serialized = new List<string>();

Console.WriteLine("Serializing values:");
foreach (var value in values)
{
    var serializedValue = serializer.SerializeValue(value);
    serialized.Add(serializedValue);
    Console.WriteLine($"  {value} → {serializedValue}");
}

Console.WriteLine("\nDeserializing values:");
foreach (var item in serialized)
{
    var deserialized = serializer.DeserializeValue(item);
    Console.WriteLine($"  {item} → {deserialized} (Type: {deserialized.GetType().Name})");
}
Console.WriteLine();

// Reflection example
Console.WriteLine("=== Reflection (Boxing Unavoidable) ===");
var point = new Point(10, 20);
RealWorldBoxingScenarios.PropertyInspector.InspectObject(point);
Console.WriteLine();

var person = new { Name = "John", Age = 30, IsActive = true };
RealWorldBoxingScenarios.PropertyInspector.InspectObject(person);
Console.WriteLine();

## Summary

### What Makes Boxing/Unboxing "Meaningful":

**Necessary Use Cases:**
- **Heterogeneous storage** - Storing different value types in one collection
- **Legacy API integration** - Working with old non-generic collections
- **Interface requirements** - When value types must implement interfaces
- **Reflection scenarios** - Property inspection, serialization
- **Event systems** - Handling different event data types
- **Plugin architectures** - Dynamic type handling

**Key Understanding:**
1. **Performance cost** - Boxing allocates memory on heap, unboxing copies data
2. **Type safety** - Unboxing requires exact type match or will throw
3. **Memory impact** - Each boxed value creates a separate object
4. **When unavoidable** - Some scenarios require boxing by design
5. **Modern alternatives** - Generics eliminate most boxing needs

### When Boxing/Unboxing Add Real Value:

**Appropriate Scenarios:**
- Configuration systems storing mixed data types
- Event systems with heterogeneous payloads
- Serialization/deserialization frameworks
- Legacy system integration
- Reflection-based operations
- Plugin or extension systems

**Safe Practices:**
- Use pattern matching (`is` operator) for safe unboxing
- Validate types before unboxing
- Prefer generic alternatives when possible
- Use `as` operator with nullable reference types
- Handle `InvalidCastException` appropriately

### Red Flags (Avoid These):

**Don't Use Boxing When:**
- **Generic alternatives exist** - Use `List<T>` instead of `ArrayList`
- **Single type collections** - No need to store int as object
- **String concatenation** - Use `StringBuilder` or interpolation
- **Performance is critical** - Boxing causes heap allocation
- **Type safety matters** - Generics provide compile-time checking

**Anti-Patterns:**
- Using `ArrayList` when `List<T>` would work
- Unnecessary boxing in string operations
- Unsafe unboxing without type checking
- Boxing in tight loops or performance-critical code
- Using `Hashtable` when `Dictionary<TKey, TValue>` is available

### Best Practices:

1. **Prefer generics** - Use `List<T>`, `Dictionary<TKey, TValue>`, etc.
2. **Safe unboxing** - Always validate types before unboxing
3. **Performance awareness** - Understand when boxing occurs
4. **Modern C# features** - Use pattern matching, nullable reference types
5. **String operations** - Use `StringBuilder` for multiple concatenations
6. **Interface design** - Use generic interfaces when possible
7. **Error handling** - Handle `InvalidCastException` gracefully
8. **Documentation** - Document when boxing is intentional

### Examples Summary:

**✅ Good Examples Shown:**
- Configuration management with mixed data types
- Safe unboxing with pattern matching and validation
- Interface implementation on structs (when necessary)
- Legacy API integration with proper error handling
- Event systems requiring heterogeneous data
- Reflection scenarios where boxing is unavoidable
- Performance measurement and optimization

**❌ Bad Examples Highlighted:**
- Using non-generic collections unnecessarily
- Unsafe unboxing without type validation
- Excessive boxing in string operations
- Performance-critical code with unnecessary boxing
- Dangerous casting that can throw exceptions

**Performance Insights:**
- Generic collections are significantly faster than non-generic ones
- `StringBuilder` avoids boxing in concatenation scenarios
- String interpolation is compiler-optimized
- Pattern matching provides safe type checking
- Each boxing operation creates a new heap object

Understanding boxing and unboxing helps you:
- Write more efficient code by avoiding unnecessary boxing
- Handle mixed-type scenarios safely and appropriately
- Debug performance issues related to memory allocation
- Make informed decisions about when boxing is acceptable
- Migrate legacy code to more efficient generic alternatives

The key is recognizing when boxing is **necessary** (heterogeneous data, legacy APIs) versus when it's **avoidable** (same-type collections, string operations) and choosing the appropriate approach for each scenario.